# Tutorial 4: Quantum Mechanics with MechanicsDSL

This tutorial explores MechanicsDSL's quantum mechanics capabilities, including tunneling,
bound states, and the hydrogen atom.

In [ ]:
from mechanics_dsl import PhysicsCompiler
from mechanics_dsl.domains.quantum import QuantumBarrier, FiniteWell, HydrogenAtom
import matplotlib.pyplot as plt
import numpy as np

## 1. Quantum Tunneling

One of the most striking quantum phenomena: particles can pass through barriers
they classically shouldn't be able to cross.

### The Physics

For a rectangular barrier of height $V_0$ and width $L$, the transmission coefficient is:

$$T = \left[1 + \frac{V_0^2 \sinh^2(\kappa L)}{4E(V_0 - E)}\right]^{-1}$$

where $\kappa = \sqrt{2m(V_0 - E)}/\hbar$

In [ ]:
# Create a rectangular barrier
barrier = QuantumBarrier(
    barrier_type='rectangular',
    V0=5.0,      # Barrier height in eV
    L=1.0,       # Barrier width in nm
    m=0.511      # Electron mass in MeV/c^2
)

# Calculate transmission for a range of energies
energies = np.linspace(0.1, 4.9, 100)  # E < V0 for tunneling
transmissions = [barrier.transmission_coefficient(E) for E in energies]

plt.figure(figsize=(10, 5))
plt.semilogy(energies, transmissions, 'b-', lw=2)
plt.xlabel('Particle Energy (eV)')
plt.ylabel('Transmission Probability')
plt.title('Quantum Tunneling Through a 5 eV Barrier')
plt.grid(True, alpha=0.3)
plt.axhline(y=1, color='r', linestyle='--', alpha=0.5, label='Classical limit')
plt.legend()
plt.show()

# Specific example
E = 3.0  # eV
T = barrier.transmission_coefficient(E)
print(f"For E = {E} eV, barrier V₀ = 5 eV, width L = 1 nm:")
print(f"Transmission probability T = {T:.4f} ({T*100:.2f}%)")
print(f"Penetration depth δ = {barrier.penetration_depth(E):.3f} nm")

## 2. Finite Square Well

A particle confined in a potential well of finite depth has discrete bound states.

### Energy Quantization

Unlike the infinite well, the number of bound states is finite and depends on
the well depth and width.

In [ ]:
# Create a finite square well
well = FiniteWell(
    V0=10.0,     # Well depth in eV
    L=2.0,       # Well width in nm
    m=0.511      # Electron mass
)

# Find bound states
bound_states = well.find_bound_states()

print(f"Finite well: V₀ = 10 eV, L = 2 nm")
print(f"Number of bound states: {len(bound_states)}\n")

for n, (E, parity) in enumerate(bound_states, 1):
    print(f"State n={n}: E = {E:.4f} eV, parity = {parity}")

In [ ]:
# Visualize wavefunctions
x = np.linspace(-3, 3, 500)  # nm

fig, axes = plt.subplots(len(bound_states), 1, figsize=(10, 3*len(bound_states)))
if len(bound_states) == 1:
    axes = [axes]

for i, (E, parity) in enumerate(bound_states):
    psi = well.wavefunction(x, E)
    psi_squared = np.abs(psi)**2
    
    axes[i].fill_between(x, 0, psi_squared, alpha=0.3)
    axes[i].plot(x, psi_squared, 'b-', lw=2)
    axes[i].axvline(x=-1, color='k', linestyle='--', alpha=0.5)
    axes[i].axvline(x=1, color='k', linestyle='--', alpha=0.5)
    axes[i].set_ylabel(f'|ψ_{i+1}|²')
    axes[i].set_title(f'State n={i+1}, E = {E:.3f} eV')
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Position (nm)')
plt.tight_layout()
plt.show()

## 3. Hydrogen Atom

The hydrogen atom is the most important exactly solvable quantum system.

### Energy Levels

$$E_n = -\frac{13.6\text{ eV}}{n^2}$$

### Spectral Series

Transitions between levels produce the famous spectral series:
- **Lyman** (n → 1): UV
- **Balmer** (n → 2): Visible
- **Paschen** (n → 3): IR

In [ ]:
# Create hydrogen atom
H = HydrogenAtom()

print("Hydrogen Atom Energy Levels")
print("=" * 40)
for n in range(1, 6):
    E = H.energy_level(n)
    print(f"n = {n}: E = {E:.4f} eV")

print(f"\nBohr radius a₀ = {H.bohr_radius:.4f} nm")
print(f"Rydberg constant R∞ = {H.rydberg_constant:.6e} m⁻¹")

In [ ]:
# Calculate spectral lines
print("\nBalmer Series (visible light transitions to n=2)")
print("=" * 50)

balmer_lines = H.spectral_series('balmer', n_max=7)

for transition, wavelength in balmer_lines:
    n_upper, n_lower = transition
    color = H.wavelength_to_color(wavelength)
    print(f"  {n_upper} → {n_lower}: λ = {wavelength:.1f} nm ({color})")

In [ ]:
# Visualize radial probability distribution
r = np.linspace(0, 20, 500)  # In units of Bohr radius

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (n, l) in zip(axes, [(1, 0), (2, 0), (2, 1)]):
    P = H.radial_probability(r, n, l)
    ax.fill_between(r, 0, P, alpha=0.3)
    ax.plot(r, P, 'b-', lw=2)
    ax.set_xlabel('r / a₀')
    ax.set_ylabel('P(r)')
    ax.set_title(f'n={n}, l={l}')
    ax.grid(True, alpha=0.3)

plt.suptitle('Radial Probability Distributions for Hydrogen', fontsize=14)
plt.tight_layout()
plt.show()

## 4. WKB Approximation

The Wentzel-Kramers-Brillouin (WKB) approximation connects quantum and classical mechanics.
It's useful for tunneling through arbitrary barrier shapes.

In [ ]:
# Define a general barrier shape
def parabolic_barrier(x, V_max=5.0, width=2.0):
    """Parabolic (inverted) barrier."""
    return np.maximum(0, V_max * (1 - (2*x/width)**2))

# Create WKB calculator for general barriers
from mechanics_dsl.domains.quantum import WKBApproximation

wkb = WKBApproximation(m=0.511)  # Electron mass

# Calculate tunneling through parabolic barrier
E = 3.0  # Particle energy in eV
T_wkb = wkb.transmission(parabolic_barrier, E, x_range=(-1, 1))

print(f"WKB transmission through parabolic barrier:")
print(f"T = {T_wkb:.4f}")

## Summary

In this tutorial, we explored:

1. **Quantum Tunneling** — Particles passing through classically forbidden barriers
2. **Finite Square Well** — Discrete bound states in a potential well
3. **Hydrogen Atom** — Energy levels and spectral series
4. **WKB Approximation** — Semiclassical tunneling calculations

## Next Steps

- **Tutorial 5**: General Relativity (Schwarzschild geodesics)
- **Tutorial 6**: Statistical Mechanics (partition functions)